In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [31]:
import keras

In [32]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential

In [33]:
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout, Activation
from keras.optimizers import Adam, RMSprop 

In [34]:
import os

In [35]:
num_classes = 2
img_rows, img_cols = 48,48
batch_size = 32

In [36]:
train_data_dir = 'D:\data\gende_classi\Training'
validation_data_dir = 'D:\data\gende_classi\Validation'

In [37]:
train_datagen = ImageDataGenerator(
                    rescale = 1./255,
                    rotation_range = 30,
                    shear_range = 0.3,
                    zoom_range = 0.3,
                    width_shift_range = 0.4,
                    height_shift_range = 0.4,
                    horizontal_flip = True,
                    fill_mode ='nearest')

In [38]:
validation_datagen = ImageDataGenerator(rescale = 1./255)

In [39]:
train_generator = train_datagen.flow_from_directory(
                                                    train_data_dir,
                                                    color_mode = 'grayscale',
                                                    target_size = (img_rows,img_cols),
                                                    batch_size = batch_size,
                                                    class_mode = 'categorical',
                                                    shuffle = True
                                                    )

Found 47009 images belonging to 2 classes.


In [40]:
validation_generator = validation_datagen.flow_from_directory(
                                                                validation_data_dir,
                                                                color_mode = 'grayscale',
                                                                target_size = (img_rows,img_cols),
                                                                batch_size = batch_size,
                                                                class_mode = 'categorical',
                                                                shuffle = True 
                                                            ) 

Found 11649 images belonging to 2 classes.


In [41]:
model = Sequential()

In [42]:
model.add(Conv2D(32,(3,3),padding='same', kernel_initializer = 'he_normal', input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer = 'he_normal', input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [43]:
model.add(Conv2D(64,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='Same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [44]:
model.add(Conv2D(128,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [45]:
model.add(Conv2D(256,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same', kernel_initializer = 'he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

In [46]:
model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [47]:
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

In [48]:
model.add(Dense(2,kernel_initializer = 'he_normal'))
model.add(Activation('softmax'))

In [49]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_9 (Conv2D)            (None, 48, 48, 32)        320       
_________________________________________________________________
activation_12 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_11 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_13 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_12 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 24, 24, 32)       

In [50]:
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau 

In [51]:
checkpoint = ModelCheckpoint('Gender_pred2.h5',
                             monitor = 'val_loss',
                             mode = 'min',
                             save_best_only = True,
                             verbose = 1)

In [52]:
earlystop = EarlyStopping(monitor = 'val_loss',
                          min_delta = 0,
                          patience = 9,
                          verbose = 1,
                          restore_best_weights = True)

In [53]:
reduce_lr =  ReduceLROnPlateau(monitor = 'val_loss',
                               factor = 0.2,
                               patience = 3,
                               verbose = 1,
                               min_delta = 0.0001)

In [54]:
callbacks = [earlystop,checkpoint,reduce_lr]

In [55]:
model.compile(optimizer =Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])

In [56]:
epochs= 25
batch_size = 32
nb_train_samples = 47009
nb_validation_samples = 11649

In [57]:
history = model.fit_generator(train_generator,
                              steps_per_epoch= nb_train_samples // batch_size,
                              epochs = epochs,
                              callbacks = callbacks,
                              validation_data = validation_generator,
                              validation_steps= nb_validation_samples // batch_size)

Epoch 1/25
1469/1469 [==============================] - 2950s 2s/step - loss: 0.6013 - accuracy: 0.6847 - val_loss: 1.2969 - val_accuracy: 0.7043

Epoch 00001: val_loss improved from inf to 1.29695, saving model to Gender_pred2.h5
Epoch 2/25
1469/1469 [==============================] - 2621s 2s/step - loss: 0.3376 - accuracy: 0.8577 - val_loss: 0.2312 - val_accuracy: 0.9367

Epoch 00002: val_loss improved from 1.29695 to 0.23120, saving model to Gender_pred2.h5
Epoch 3/25
1469/1469 [==============================] - 3586s 2s/step - loss: 0.2807 - accuracy: 0.8870 - val_loss: 0.1985 - val_accuracy: 0.9179

Epoch 00003: val_loss improved from 0.23120 to 0.19849, saving model to Gender_pred2.h5
Epoch 4/25
1469/1469 [==============================] - 3143s 2s/step - loss: 0.2550 - accuracy: 0.8987 - val_loss: 0.1040 - val_accuracy: 0.9451

Epoch 00004: val_loss improved from 0.19849 to 0.10397, saving model to Gender_pred2.h5
Epoch 5/25
1469/1469 [==============================] - 6279s 4s